In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install fair-esm

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from Bio import SeqIO
import esm
import numpy as np
from sklearn.metrics import f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import random

# import os.path
# os.chdir("/kaggle/working")

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
class ProteinSequenceDataset(Dataset):
    def __init__(self, fasta_file, arg_dict, alphabet, max_samples_per_class=1000):
        self.alphabet = alphabet
        self.sequences, self.labels, self.ids= self.load_sequences(fasta_file, arg_dict, max_samples_per_class)

    def encode_sequence(self, sequence, alphabet):
        tokens = alphabet.encode(sequence)
        return torch.tensor(tokens, dtype=torch.long)
    
    def load_sequences(self, fasta_file, arg_dict, max_samples_per_class):
        temp_sequences = {label: [] for label in arg_dict.values()}  # Initialize with integers
        sequences, labels, ids = [], [], []
        
        for record in SeqIO.parse(fasta_file, "fasta"):
            if 'FEATURES' in record.description:
                label = record.description.split('|')[3]
                label_idx = arg_dict.get(label, arg_dict['nonarg'])
            else:
                label_idx = arg_dict['nonarg']

            seq_encoded = self.encode_sequence(str(record.seq), self.alphabet)
            temp_sequences[label_idx].append((seq_encoded, label_idx, record.id))
        
        for label_idx, items in temp_sequences.items():
            if max_samples_per_class is not None and len(items) > max_samples_per_class:
                items = random.sample(items, max_samples_per_class)
            for seq_encoded, label_idx, record_id in items:
                sequences.append(seq_encoded)
                labels.append(label_idx)
                ids.append(record_id)

        return sequences, labels, ids
    
    def one_hot_encode(self, sequence):
        aa_to_int = {aa: i for i, aa in enumerate('ARNDCEQGHILKMFPSTWYV')}
        one_hot = torch.zeros(len(sequence), 20)  # 20 amino acids
        for i, aa in enumerate(sequence):
            if aa in aa_to_int:
                one_hot[i, aa_to_int[aa]] = 1.0
        return one_hot
    
    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.ids[idx]

arg_dict = {
    'aminoglycoside': 0, 'macrolide-lincosamide-streptogramin': 1, 'polymyxin': 2,
    'fosfomycin': 3, 'trimethoprim': 4, 'bacitracin': 5, 'quinolone': 6, 'multidrug': 7,
    'chloramphenicol': 8, 'tetracycline': 9, 'rifampin': 10, 'beta_lactam': 11,
    'sulfonamide': 12, 'glycopeptide': 13, 'nonarg': 14
}

from torch.nn.utils.rnn import pad_sequence
# dealing with sequences of different lengths
def collate_batch(batch):
    sequences, labels, ids = zip(*batch)
    # pad sequences to have the same length
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    labels = torch.LongTensor(labels)
    return sequences_padded, labels, ids


In [ ]:
class ESMClassifier(nn.Module):
    def __init__(self, esm_model, num_classes):
        super().__init__()
        self.esm_model = esm_model
        self.classifier = nn.Linear(320, num_classes)

    def forward(self, x):
        results = self.esm_model(x, repr_layers=[6])
        representations = results["representations"][6]
        pooled_representations = representations.mean(dim=1)
        return self.classifier(pooled_representations)

In [ ]:
def train_model(classifier, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_val_f1 = -1
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        classifier.train()
        total_loss = 0
        for sequences, labels, _ in train_loader:

            sequences, labels = sequences.to(device), labels.to(device)

            predictions = classifier(sequences)
            loss = criterion(predictions, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        val_f1 = evaluate_model(classifier, val_loader)
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(classifier.state_dict(), 'model.pth')
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}, Val F1: {val_f1}')


def evaluate_model(classifier, data_loader):
    classifier.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for sequences, labels, _ in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = classifier(sequences)
            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    f1 = f1_score(all_targets, all_predictions, average='macro')
#     print('F1 Score:', f1)
    return f1

def generate_predictions_and_save(classifier, test_loader, file_path='submission.csv'):
    classifier.eval()
    sequence_ids = []
    predictions = []
    with torch.no_grad():
        for sequences, labels, ids in test_loader:
            print(len(sequences))
            sequences = sequences.to(device)
            outputs = classifier(sequences)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
            sequence_ids.extend(ids)

    df = pd.DataFrame({'id': sequence_ids, 'label': predictions})
    df.to_csv(file_path, index=False)

In [ ]:
# model_path = "/kaggle/input/model-pth/model.pth"
model_path = "/kaggle/input/model/model.pth"
esm_path = "/kaggle/input/esm2-pretrained/esm2_t6_8M_UR50D.pt"
data_path = "/kaggle/input/aist4010-spring2024-a2/data"
num_epochs = 15
lr=0.0000005
batch_size = 4

model, alphabet = esm.pretrained.load_model_and_alphabet(esm_path)
model.to(device)
model.eval()

train_dataset = ProteinSequenceDataset(data_path+'/train.fasta', arg_dict, alphabet)
val_dataset = ProteinSequenceDataset(data_path+'/val.fasta', arg_dict, alphabet, max_samples_per_class = None)
test_dataset = ProteinSequenceDataset(data_path+'/test.fasta', arg_dict, alphabet, max_samples_per_class = None)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

classifier = ESMClassifier(model, len(arg_dict))
classifier = classifier.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=lr)
classifier.load_state_dict(torch.load(model_path))



In [ ]:

# train_model(classifier, train_loader, val_loader, criterion, optimizer, num_epochs)

In [ ]:
# classifier.load_state_dict(torch.load("/kaggle/working/model.pth"))
# generate_predictions_and_save(classifier, test_loader)

# LORA (Microsoft)

In [11]:
!pip install loralib

In [20]:
import loralib as lora

class ESMClassifier(nn.Module):
    def __init__(self, esm_model, num_classes, lora_config=None):
        super().__init__()
        self.esm_model = esm_model
        if lora_config:
            self.classifier = lora.Linear(320, # esm2_t6_8M_UR50D Embedding Dim: 320 
                                          num_classes, 
                                          r=lora_config['r'], 
                                          lora_alpha=lora_config['lora_alpha'], 
                                          lora_dropout=lora_config['lora_dropout'])
        else:
            self.classifier = nn.Linear(esm_model.classifier.in_features, num_classes)


    def forward(self, x):
        results = self.esm_model(x, repr_layers=[6]) #esm2_t6_8M_UR50D layers = 6
        representations = results["representations"][6]
        pooled_representations = representations.mean(dim=1)
        return self.classifier(pooled_representations)
    

# model_path = "/kaggle/input/model-pth/model.pth"
model_path = "/kaggle/input/model/model.pth"
model_path = "/kaggle/working/model_base_lora.pth"
esm_path = "/kaggle/input/esm2-pretrained/esm2_t6_8M_UR50D.pt"
data_path = "/kaggle/input/aist4010-spring2024-a2/data"

batch_size = 10

model, alphabet = esm.pretrained.load_model_and_alphabet(esm_path)
model.to(device)
model.eval()

train_dataset = ProteinSequenceDataset(data_path+'/train.fasta', arg_dict, alphabet)
val_dataset = ProteinSequenceDataset(data_path+'/val.fasta', arg_dict, alphabet, max_samples_per_class = None)
test_dataset = ProteinSequenceDataset(data_path+'/test.fasta', arg_dict, alphabet, max_samples_per_class = None)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)


In [24]:
model_path = "/kaggle/input/model/model.pth"
model_path = "/kaggle/working/model_base_lora.pth"

lora_config = {
    'r': 8,
    'lora_alpha': 24,
    'lora_dropout': 0.5,
}
num_epochs = 100
lr=0.0001



classifier = ESMClassifier(model, len(arg_dict), lora_config=lora_config)
classifier = classifier.to(device)
classifier.load_state_dict(torch.load(model_path), strict=False)

lora.mark_only_lora_as_trainable(classifier)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=lr)



def train_model(classifier, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_val_f1 = -1
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        classifier.train()
        total_loss = 0
        for sequences, labels, _ in train_loader:

            sequences, labels = sequences.to(device), labels.to(device)

            predictions = classifier(sequences)
            loss = criterion(predictions, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        val_f1 = evaluate_model(classifier, val_loader)
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(classifier.state_dict(), 'model_base_lora.pth')
            torch.save(lora.lora_state_dict(classifier), 'lora_model.pth')

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}, Val F1: {val_f1}')


def evaluate_model(classifier, data_loader):
    classifier.eval()
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for sequences, labels, _ in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = classifier(sequences)
            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    f1 = f1_score(all_targets, all_predictions, average='macro')
#     print('F1 Score:', f1)
    return f1

def generate_predictions_and_save(classifier, test_loader, file_path='submission.csv'):
    classifier.eval()
    sequence_ids = []
    predictions = []
    with torch.no_grad():
        for sequences, labels, ids in test_loader:
            outputs = classifier(sequences)            sequences = sequences.to(device)

            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
            sequence_ids.extend(ids)

    df = pd.DataFrame({'id': sequence_ids, 'label': predictions})
    df.to_csv(file_path, index=False)

train_model(classifier, train_loader, val_loader, criterion, optimizer, num_epochs)


Epochs:   1%|          | 1/100 [01:42<2:49:29, 102.73s/it]

Epoch 1/100, Loss: 0.012277000624962658, Val F1: 0.9221905176510666


Epochs:   2%|▏         | 2/100 [03:24<2:47:03, 102.28s/it]

Epoch 2/100, Loss: 0.012562273693557422, Val F1: 0.9221304016881108


Epochs:   3%|▎         | 3/100 [05:06<2:45:12, 102.19s/it]

...

Epoch 98/100, Loss: 0.009874432329544248, Val F1: 0.9233211491619868


Epochs:  99%|█████████▉| 99/100 [2:48:54<01:42, 102.56s/it]

Epoch 99/100, Loss: 0.008118097492718248, Val F1: 0.9226273980529996


Epochs: 100%|██████████| 100/100 [2:50:35<00:00, 102.36s/it]

Epoch 100/100, Loss: 0.0079377332543038, Val F1: 0.9217151394802078


In [ ]:

classifier.load_state_dict(torch.load('model_base_lora.pth'), strict=False)
classifier.load_state_dict(torch.load('lora_model.pth'), strict=False)

generate_predictions_and_save(classifier, test_loader)